In [1]:
from jinja2 import DictLoader, Environment, FileSystemLoader
import os
PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)
def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)

In [2]:
import xml.etree.ElementTree as etree
source_xmlns = "{http://docs.oasis-open.org/xmile/ns/XMILE/v1.0}" 
source_xmlns_isee = "{http://iseesystems.com/XMILE}"

In [3]:
matlab_file = 'goodwin-minsky-with-names.m'
template_xmile = 'templates/template-xmile.xml'

In [ ]:
# Parsear archivo Matlab
# Generar context para pasar al template de xmile